In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
#Now we want to index it
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question': 3.0, 'section': 0.5}

result = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    num_results=5
)

In [10]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
# from openai import OpenAI
from google import genai

In [12]:
# client = OpenAI()
client = genai.Client()

In [13]:
# response = client.chat.completions.create(
#     model='gpt-4o',
#     messages=[{"role": "user", "content": q}]
# )
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=q
)

In [14]:
print(response.text)

It really depends entirely on **where the course is offered** and **how far along it is**.

Here's a breakdown of common scenarios:

1.  **University or College Course (for credit):**
    *   **Likelihood:** Often **difficult, but sometimes possible within a very short grace period.** Most universities have strict add/drop deadlines, usually within the first week or two of the semester. After that, you'd typically need special permission from the department head or even the dean, which is rare and usually only for extenuating circumstances.
    *   **What to do:**
        *   Check the academic calendar for add/drop deadlines.
        *   Contact the Registrar's Office or the specific department offering the course immediately.
        *   You might be able to audit the course or take it next semester.

2.  **Online Learning Platforms (Coursera, edX, FutureLearn, etc. - cohort-based):**
    *   **Likelihood:** **Sometimes yes, sometimes no.** If it's a "live" cohort with specific start

In [15]:
#We want the LLM to put the document content in the context of the answer
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FQA database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [16]:
context = ""

for doc in result:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
# response = client.chat.completions.create(
#     model='gpt-4o',
#     messages=[{"role": "user", "content": prompt}]
# )

# response.choices[0].message.content
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
)

print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects.


In [ ]:
def search(query):
    